In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:

import numpy as np
import matplotlib.pyplot as plt
from tensorflow import data as tf_data
import keras
from keras import models ,layers
from warnings import filterwarnings
filterwarnings(action='ignore')
import os 
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tqdm import tqdm


image_size = (299, 299)
batch_size = 32

train_ds = keras.utils.image_dataset_from_directory(
    '../Desktop/Data/train_1',
    labels="inferred",
    label_mode="categorical",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

val_ds = keras.utils.image_dataset_from_directory(
    '../Desktop/Data/test_1',
    labels="inferred",
    label_mode="categorical",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

test_ds = keras.utils.image_dataset_from_directory(
    '../Desktop/Data/test_2',
    labels="inferred",
    label_mode="categorical",
    seed=1337,
    image_size=image_size,
    batch_size=1,
)







2024-04-22 10:58:06.537021: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 10:58:06.588269: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 10:58:07.519569: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 60580 files belonging to 2 classes.


2024-04-22 10:58:11.708974: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 13231 files belonging to 2 classes.
Found 15620 files belonging to 2 classes.


In [2]:
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)
test_ds = test_ds.prefetch(tf_data.AUTOTUNE)

In [3]:
xception_model=load_model("save_xception_model_at_4.keras")

epochs = 10

callbacks = [
    keras.callbacks.ModelCheckpoint('./2204save_xception_model_at_{epoch}.keras',
                                    monitor="val_loss",
                                    mode="min",
                                    save_freq="epoch"),
    keras.callbacks.EarlyStopping(monitor="val_loss",
                                  patience=2,
                                  mode="min"),
    keras.callbacks.CSVLogger('2204_xception_model_fit_logs.csv',append=False)
]

xception_model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)
history=xception_model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)



Epoch 1/10
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - acc: 0.7818 - loss: 0.4716 

2024-04-22 17:12:55.339961: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 34330240 bytes after encountering the first element of size 34330240 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


1894/1894 ━━━━━━━━━━━━━━━━━━━━ 23413s 12s/step - acc: 0.7818 - loss: 0.4715 - val_acc: 0.5958 - val_loss: 1.2121
Epoch 2/10
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 22691s 12s/step - acc: 0.9410 - loss: 0.1576 - val_acc: 0.6461 - val_loss: 1.0488
Epoch 3/10
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 22723s 12s/step - acc: 0.9642 - loss: 0.1008 - val_acc: 0.6390 - val_loss: 1.4601
Epoch 4/10
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 22432s 12s/step - acc: 0.9744 - loss: 0.0760 - val_acc: 0.6411 - val_loss: 1.5543


In [4]:
result=xception_model.evaluate(test_ds)
print("Accuracy : ",result[1])

15428/15620 ━━━━━━━━━━━━━━━━━━━━ 21s 113ms/step - acc: 0.5528 - loss: 1.9986

KeyboardInterrupt: 